### 1. Install dependencies

In [126]:
import os
from dotenv import load_dotenv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


### 2. FX scraper

In [129]:
url = "https://www.dbs.com/in/treasures/rates-online/foreign-currency-foreign-exchange.page"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

In [132]:
date_detail = driver.find_element(By.ID, "mainCurrency").text
elements = date_detail.split(' ')

date = elements[3]
time = elements[-1]

In [3]:
# Header Names
header = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/thead/tr/th")

col_names = list()

for head in header:
    col_names.append(head.text)

print(col_names)


['Currency', 'Selling TT', 'Selling Cash', 'Buying TT', 'Buying Cash']


In [4]:
# Locators
col = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr")
row = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr[1]/td")

num_row = len(row) #5
num_col = len(col) #10

results = []


for i in range (1, num_col + 1):
    result = []
    for j in range (1, num_row + 1):
        d = driver.find_element(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr["+str(i)+"]/td["+str(j)+"]").text
        result.append(d)
    results.append(result)

In [6]:
df = pd.DataFrame(data=results, columns=col_names)

df["Date"] = date
df["Time"] = time

print(df)

                      Currency Selling TT Selling Cash Buying TT Buying Cash  \
0                    US Dollar    81.2468      81.2468   78.0607     78.0607   
1             Singapore Dollar    59.4800      59.4800   56.8625     56.8625   
2            Australian Dollar    57.9094      57.9094   55.3610     55.3610   
3                  Swiss Franc    86.5179      86.5179   82.7106     82.7106   
4                         Euro    84.0371      84.0371   80.3389     80.3389   
5                British Pound    99.3098      99.3098   94.9394     94.9394   
6              Canadian Dollar    63.8483      63.8483   61.0385     61.0385   
7             Hong Kong Dollar    10.3947      10.3947    9.9373      9.9373   
8                 Japanese Yen     0.6111       0.6111    0.5842      0.5842   
9  United Arab Emirates Dirham    22.1744      22.1744   21.1986     21.1986   

         Date     Time  
0  12/08/2022  11:45AM  
1  12/08/2022  11:45AM  
2  12/08/2022  11:45AM  
3  12/08/2022  11:4

# 3. PDF File

In [16]:
pdf = "/home/oem/Documents/coding/personal/Reits-Analytics/data/Ascendas-Reit-Annual-Report-2021.pdf"

import tabula
import pandas as pd

tables = tabula.read_pdf(pdf, pages=[72])
df = tables[0]
df = df.rename(columns={"Unnamed: 0": "num", 
                "Unnamed: 1": "property", 
                "Business & Science Park Properties": "development_cost",	
                "Unnamed: 2": "valuations_(SGD:Mil)",
                "Unnamed: 3": "gfa_(Sqm)",
                "Unnamed: 4": "nla_(Sqm)",
                "Unnamed: 5": "address",
                "Unnamed: 6": "gross_revenue_(SGD:Mil)",
                "Unnamed: 7": "occupancy_rate"})
# remove first rows
df = df.iloc[4:]
# split out completion date parsed incorrectly
def split_completion_date(text):
    if isinstance(text, str):
        x = text[:9]
        return x

df["completion_date"] = df["development_cost"].apply(lambda x: split_completion_date(x))
# split out development cost parsed incorrectly
def split_development_cost(text):
    if isinstance(text, str) and len(text) > 11:
        amt = text[10:]
        return amt

df["development_cost"] = df["development_cost"].apply(lambda x: split_development_cost(x))
df = df.dropna(how="any", subset=["development_cost", "completion_date"])

charlist = ["#", "*", "^"]
for char in charlist:
    df["property"] = df["property"].str.replace(char, "", regex=True)


# 4. Geographical API

In [17]:
from dotenv import load_dotenv 
import os
import numpy as np
import requests

query = "dakota breeze"

class google_api:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv("GOOGLE_KEY")

    def return_lat_long(self, query, country):
        query = self.custom_query(query, country)
        path = f"https://maps.googleapis.com/maps/api/geocode/json?address={query}?,+CA&key={self.api_key}"
        response = requests.get(path)
        jsonResponse = response.json()
        if jsonResponse["status"]=="OK":
            lat = jsonResponse["results"][0]["geometry"]["location"]["lat"]
            long = jsonResponse["results"][0]["geometry"]["location"]["lng"]
            return lat, long
        return None

    def custom_query(self, query, country):
        loc_dict = {"1, 3 & 5 Changi Business Park Crescent": "3 Changi Business Park Crescent", 
                    "77 & 79 Science Park Drive": "77 Science Park Drive",
                    "41, 45 & 51 Science Park Road": "51 Science Park Road",
                    "12, 14 & 16 Science Park Drive": "16 Science Park Drive"}
        if query in loc_dict:
            query = loc_dict.get(query)
        query = query+f", {country}"
        query = query.replace(" ", "%20")

        return query

In [18]:
df["address_cord"] = df["address"].apply(lambda x: google_api().return_lat_long(x, "Singapore"))
df["property_cord"] = df["property"].apply(lambda x: google_api().return_lat_long(x, "Singapore"))

df["lat_lng"] = np.where(df["address_cord"].isna(),df["property_cord"],df["address_cord"])

df["lattitude"] = df["lat_lng"].apply(lambda x: x[0])
df["longitude"] = df["lat_lng"].apply(lambda x: x[1])

df.drop(columns=["address_cord", "property_cord", "lat_lng"], inplace=True)


In [19]:
df.head()

,num,property,development_cost,valuations_(SGD:Mil),gfa_(Sqm),nla_(Sqm),address,gross_revenue_(SGD:Mil),occupancy_rate,completion_date,lattitude,longitude
5,1.0,Neuros & Immunos,125.6,143.0,"36,931","26,035",8/8A Biomedical Grove,21.0,98.4%,31 Mar 11,1.302090,103.792781
6,2.0,Nexus @one-north,181.3,202.8,"25,511","20,669",1 & 3 Fusionopolis Link,15.0,93.4%,04 Sep 13,1.298802,103.789988
7,3.0,Nucleos,289.0,346.1,"46,174","37,365",21 Biopolis Road,30.6,98.4%,11 Dec 19,1.301487,103.792140
8,4.0,Galaxis,697.5,733.6,"68,835","60,709",1 & 3 Fusionopolis Place,25.7,97.0%,30 Jun 21,1.299864,103.788399
9,5.0,Grab Headquarters,184.6,191.5,"42,290","42,290",1 & 3 Media Close,4.6,100.0%,30 Jul 21,1.291818,103.793046


In [20]:
floattypes = ["development_cost", "valuations_(SGD:Mil)", "gross_revenue_(SGD:Mil)"]

def to_numeric(number):
    try:
        return float(number)
    except:
        return 0

for each in floattypes:
    df[each] = df[each].apply(lambda x: to_numeric(x))

In [21]:
df.head(5)

,num,property,development_cost,valuations_(SGD:Mil),gfa_(Sqm),nla_(Sqm),address,gross_revenue_(SGD:Mil),occupancy_rate,completion_date,lattitude,longitude
5,1.0,Neuros & Immunos,125.6,143.0,"36,931","26,035",8/8A Biomedical Grove,21.0,98.4%,31 Mar 11,1.302090,103.792781
6,2.0,Nexus @one-north,181.3,202.8,"25,511","20,669",1 & 3 Fusionopolis Link,15.0,93.4%,04 Sep 13,1.298802,103.789988
7,3.0,Nucleos,289.0,346.1,"46,174","37,365",21 Biopolis Road,30.6,98.4%,11 Dec 19,1.301487,103.792140
8,4.0,Galaxis,697.5,733.6,"68,835","60,709",1 & 3 Fusionopolis Place,25.7,97.0%,30 Jun 21,1.299864,103.788399
9,5.0,Grab Headquarters,184.6,191.5,"42,290","42,290",1 & 3 Media Close,4.6,100.0%,30 Jul 21,1.291818,103.793046


# 5. Vizualization

In [22]:
import plotly.express as px
from dotenv import load_dotenv 
import os
import plotly.graph_objects as go

load_dotenv()
api_key = os.getenv("MAPBOX_KEY")

fig = px.scatter_mapbox(df, 
                        title="Business & Science Parks", 
                        lon="longitude", 
                        lat="lattitude", 
                        color="property", 
                        size="gross_revenue_(SGD:Mil)", 
                        zoom=9, 
                        height=500,
                        width=900) 

fig.update_layout(mapbox_style="mapbox://styles/mapbox/navigation-night-v1", mapbox_accesstoken=api_key)
fig.update_layout(autosize=True, margin={"r": 10, "t": 50, "l": 10, "b": 10})
fig.update_layout(mapbox_bounds={"west": 0, "east": 1000, "south": 500, "north": 0})

fig.show()
